In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import scipy.stats as stats
import pickle

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, learning_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.base import BaseEstimator, ClassifierMixin

# Lectura de los datos

In [3]:
train=pd.read_csv('../data_processed/clean/train_data.csv')

In [4]:
test=pd.read_csv('../data_processed/clean/test_data.csv')

In [6]:
X_train = train.drop('Class', axis=1)
y_train = train['Class']
X_test = test.drop('Class', axis=1)
y_test = test['Class']

# Prueba de distintos modelos

In [7]:
# Función para calcular métricas tras la predicción
def classification_metrics(y_test, y_pred, name, model=np.NaN, roc_bool=True):
    """
    Calcula y muestra diversas métricas de clasificación tras realizar una predicción.

    Parámetros:
    -----------
    y_test : array-like
        Valores verdaderos de la variable objetivo (labels) del conjunto de test.
    y_pred : array-like
        Valores predichos por el modelo.
    name : str
        Nombre o descripción del modelo, utilizado para etiquetar la salida.
    model : object, optional
        Modelo utilizado para hacer la predicción. Se utiliza para calcular la métrica
        ROC-AUC si el modelo soporta la función `predict_proba()`. Por defecto, es np.NaN.
    roc_bool : bool, optional
        Variable para pintar (o no) la curva ROC-AUC.

    Métricas calculadas:
    --------------------
    - Accuracy: Precisión del modelo.
    - Matriz de confusión: Representación de los aciertos y errores del modelo.
    - Reporte de clasificación: Incluye precisión, recall, F1-score, etc.
    - ROC-AUC: Se calcula solo si el modelo soporta `predict_proba()`.
    
    Devuelve:
    ---------
    data_metrics : pd.DataFrame
        Dataframe con la información de métricas del modelo evaluado en la función.
    """
    
    # Calcular la precisión (accuracy)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Mostrar resultados
    print(f"\n----- {name} -----")
    print(f"Accuracy: {accuracy:.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    metrics_summary = {'Model': [], 'Accuracy': [], 'Precision': [], 'Recall': [], 'F1-Score': [], 'ROC-AUC': []}

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Calcular y mostrar ROC-AUC si el modelo lo permite y se pide
    if roc_bool:
        if hasattr(model, "predict_proba"):
            y_proba = model.predict_proba(X_test)[:,1]
            roc_auc = roc_auc_score(y_test, y_proba)
            fpr, tpr, _ = roc_curve(y_test, y_proba)
            print(f"ROC-AUC: {roc_auc:.4f}")
            
            plt.figure(figsize=(8, 6))
            plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.4f})')
            plt.plot([0, 1], [0, 1], 'k--', label='Aleatorio')
            plt.xlabel('Tasa de Falsos Positivos')
            plt.ylabel('Tasa de Verdaderos Positivos')
            plt.title('Curva ROC')
            plt.legend()
            plt.show()
        
        else:
            print(f"ROC-AUC: No disponible para {name}")

     # Almacenar los resultados en el diccionario
    metrics_summary['Model'].append(name)
    metrics_summary['Accuracy'].append(accuracy)
    metrics_summary['Precision'].append(precision)
    metrics_summary['Recall'].append(recall)
    metrics_summary['F1-Score'].append(f1)
    if roc_bool == True & hasattr(model, "predict_proba"):
        metrics_summary['ROC-AUC'].append(roc_auc)
    else:
        metrics_summary['ROC-AUC'].append(np.NaN)
    
    return pd.DataFrame(metrics_summary)

# Función para entrenar y evaluar modelos
def evaluate_models(models, X_train, X_test, y_train, y_test):
    """
    Entrena y evalúa múltiples modelos de clasificación en un conjunto de datos de prueba.

    Parámetros:
    -----------
    models : dict
        Un diccionario donde las claves son los nombres de los modelos (como cadenas de texto) 
        y los valores son instancias de los modelos de clasificación.
    X_train : array-like
        Características (features) del conjunto de entrenamiento.
    X_test : array-like
        Características (features) del conjunto de prueba.
    y_train : array-like
        Valores verdaderos de la variable objetivo (labels) del conjunto de entrenamiento.
    y_test : array-like
        Valores verdaderos de la variable objetivo (labels) del conjunto de prueba.

    Funcionalidad:
    --------------
    - Entrena cada modelo en `models` usando el conjunto de entrenamiento `X_train` y `y_train`.
    - Predice los valores de la variable objetivo en `X_test` para cada modelo.
    - Calcula y muestra métricas de clasificación (incluyendo accuracy, matriz de confusión, 
      reporte de clasificación y ROC-AUC si está disponible) utilizando la función 
      `classification_metrics`.

    Devuelve:
    ---------
    data_metrics : pd.DataFrame
        Dataframe con la información de métricas de cada modelo evaluado en la función.
    """

    data_metrics = pd.DataFrame()
    
    # Iterar sobre todos los modelos definidos
    for name, model in models.items():
       
        # Entrenar el modelo
        model.fit(X_train, y_train)
        
        # Predecir en los datos de prueba
        y_pred = model.predict(X_test)

        # Sacar métricas
        df_met_model = classification_metrics(y_test, y_pred, name, model)

        # Métricas training
        print("--- MÉTRICAS CON ENTRENAMIENTO: ---")
        df_met_model_train = classification_metrics(y_train, model.predict(X_train), name, model, roc_bool=False)

        # Renombrar columnas de df_met_model_train para concatenar
        df_met_model_train = df_met_model_train.drop(columns=["ROC-AUC"])
        df_met_model_train.rename(columns={col: col + '_train' for col in df_met_model_train.columns if col != 'Model'}, inplace=True)
        
        # Juntar los dos dataframes en uno
        df_met_model = df_met_model.merge(df_met_model_train, left_on='Model', right_on='Model')
        
        if data_metrics.shape[0] > 0:
            data_metrics = pd.concat([data_metrics, df_met_model])
        else:
            data_metrics = df_met_model

    return data_metrics.reset_index(drop=True).loc[:, ['Model', 'Accuracy_train', 'Accuracy', 'Precision_train', 'Precision', 'Recall_train', 'Recall', 'F1-Score_train', 'F1-Score', 'ROC-AUC']]